In [1]:
!pip install tfts --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 1.7 MB/s eta 0:00:00


In [2]:
import logging
from typing import List, Optional, Union
import numpy as np
import pandas as pd
import tensorflow as tf
from tfts import AutoModel, AutoConfig, KerasTrainer

2025-05-15 03:03:43.156887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747278223.380152      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747278223.444042      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1747278239.457192      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [3]:
class CFG:
    input_dir = "/kaggle/input/china-vehicle-sales-data/china_vehicle_sales_data.csv"
    train_sequence_length = 12
    predict_sequence_length = 3


In [4]:
data = pd.read_csv(CFG.input_dir)

data

,Date,province,provinceId,popularity,model,bodyType,salesVolume
0,201601,Shanghai,310000,1479,3c974920a76ac9c1,SUV,292
1,201601,Yunnan,530000,1594,3c974920a76ac9c1,SUV,466
2,201601,Inner Mongolia,150000,1479,3c974920a76ac9c1,SUV,257
3,201601,Beijing,110000,2370,3c974920a76ac9c1,SUV,408
4,201601,Sichuan,510000,3562,3c974920a76ac9c1,SUV,610
...,...,...,...,...,...,...,...
43291,201708,Inner Mongolia,150000,9716,32d3069d17aa47c2,MPV,58
43292,201709,Inner Mongolia,150000,9128,32d3069d17aa47c2,MPV,74
43293,201710,Inner Mongolia,150000,8700,32d3069d17aa47c2,MPV,48
43294,201711,Inner Mongolia,150000,7284,32d3069d17aa47c2,MPV,48


In [5]:
# https://github.com/hongyingyue/Vehicle-sales-predictor/blob/main/vehicle_ml/feature/ts_feature.py

logger = logging.getLogger(__name__)

def add_lagging_feature(
    data: pd.DataFrame,
    groupby_column: Union[str, List[str]],
    value_columns: List[str],
    lags: List[int],
    feature_columns: Optional[List[str]] = None,
):
    # note that the data should be sorted by time already
    # the lagging feature could be further developed use f1 - f1_lag, or f1 / f1_lag

    if not isinstance(groupby_column, (str, list)):
        raise TypeError(f"'groupby_column' must be a string or a list of strings, but got {type(groupby_column)}.")

    if not isinstance(value_columns, (list, tuple)):
        raise TypeError(f"'value_columns' must be a list of strings, but got {type(value_columns)}.")

    feature_columns: List[str] = feature_columns if feature_columns is not None else []
    for column in value_columns:
        if column not in data.columns:
            raise ValueError(f"Value column '{column}' not found in DataFrame.")

        for lag in lags:
            feature_col_name = f"{column}_lag{lag}"
            feature_columns.append(feature_col_name)
            logger.debug(
                f"Creating lagging feature: {feature_col_name} for column '{column}' with lag {lag} and groupby '{groupby_column}'."
            )
            data[feature_col_name] = data.groupby(groupby_column)[column].shift(lag)
    return data

In [6]:
feature_columns = []

data = add_lagging_feature(data, groupby_column=["provinceId", "model"], value_columns=["salesVolume"], lags=list(range(1, 12)), feature_columns=feature_columns)

data

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Date,province,provinceId,popularity,model,bodyType,salesVolume,salesVolume_lag1,salesVolume_lag2,salesVolume_lag3,salesVolume_lag4,salesVolume_lag5,salesVolume_lag6,salesVolume_lag7,salesVolume_lag8,salesVolume_lag9,salesVolume_lag10,salesVolume_lag11
0,201601,Shanghai,310000,1479,3c974920a76ac9c1,SUV,292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201601,Yunnan,530000,1594,3c974920a76ac9c1,SUV,466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201601,Inner Mongolia,150000,1479,3c974920a76ac9c1,SUV,257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201601,Beijing,110000,2370,3c974920a76ac9c1,SUV,408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201601,Sichuan,510000,3562,3c974920a76ac9c1,SUV,610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43291,201708,Inner Mongolia,150000,9716,32d3069d17aa47c2,MPV,58,67.0,62.0,36.0,35.0,54.0,31.0,55.0,44.0,39.0,40.0,47.0
43292,201709,Inner Mongolia,150000,9128,32d3069d17aa47c2,MPV,74,58.0,67.0,62.0,36.0,35.0,54.0,31.0,55.0,44.0,39.0,40.0
43293,201710,Inner Mongolia,150000,8700,32d3069d17aa47c2,MPV,48,74.0,58.0,67.0,62.0,36.0,35.0,54.0,31.0,55.0,44.0,39.0
43294,201711,Inner Mongolia,150000,7284,32d3069d17aa47c2,MPV,48,48.0,74.0,58.0,67.0,62.0,36.0,35.0,54.0,31.0,55.0,44.0


In [7]:
grouped_sequence = data.groupby(["provinceId", "model"]).apply(
    lambda x: x.sort_values('Date')[["salesVolume", "salesVolume_lag1", "salesVolume_lag2", "salesVolume_lag3"]].to_numpy()
)

data_3d = np.stack(grouped_sequence.values)

data_3d

/tmp/ipykernel_19/3102104721.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_sequence = data.groupby(["provinceId", "model"]).apply(


array([[[ 799.,   nan,   nan,   nan],
        [ 424.,  799.,   nan,   nan],
        [ 733.,  424.,  799.,   nan],
        ...,
        [ 544.,  659.,  630.,  670.],
        [ 647.,  544.,  659.,  630.],
        [ 640.,  647.,  544.,  659.]],

       [[ 135.,   nan,   nan,   nan],
        [  57.,  135.,   nan,   nan],
        [ 160.,   57.,  135.,   nan],
        ...,
        [ 105.,  201.,  120.,  135.],
        [ 148.,  105.,  201.,  120.],
        [ 112.,  148.,  105.,  201.]],

       [[ 872.,   nan,   nan,   nan],
        [ 197.,  872.,   nan,   nan],
        [ 494.,  197.,  872.,   nan],
        ...,
        [ 152.,  170.,  181.,  159.],
        [ 213.,  152.,  170.,  181.],
        [ 226.,  213.,  152.,  170.]],

       ...,

       [[ 181.,   nan,   nan,   nan],
        [  60.,  181.,   nan,   nan],
        [ 111.,   60.,  181.,   nan],
        ...,
        [ 330.,  297.,  252.,  199.],
        [ 178.,  330.,  297.,  252.],
        [ 185.,  178.,  330.,  297.]],

       [[1023.,

In [8]:
from tensorflow.keras.utils import Sequence


class TimeDataset(Sequence):
    def __init__(self, data, train_sequence_length, predict_sequence_length, batch_size: int = 64):
        self.data = data
        self.train_seq_len = train_sequence_length
        self.pred_seq_len = predict_sequence_length
        self.batch_size = batch_size

        self.num_ids = data.shape[0]
        self.max_seq_len = data.shape[1]
        self.feature_dim = data.shape[2]

        self.samples_per_id = self.max_seq_len - self.train_seq_len - self.pred_seq_len + 1
        self.total_samples = self.num_ids * self.samples_per_id

        # Precompute all valid (id, start_idx) pairs
        self.indices = [
            (i, j)
            for i in range(self.num_ids)
            for j in range(self.samples_per_id)
        ]
    
    def __getitem__(self, index):
        # batch-wise item        
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        x_batch = []
        y_batch = []

        for id_idx, start_idx in batch_indices:
            x = self.data[id_idx, start_idx:start_idx + self.train_seq_len, 1:]
            y = self.data[id_idx, start_idx + self.train_seq_len:start_idx + self.train_seq_len + self.pred_seq_len, 0]
            x_batch.append(x)
            y_batch.append(y)

        return np.nan_to_num(np.array(x_batch)), np.nan_to_num(np.array(y_batch))
    
    def __len__(self):
        # depends on how many samples you want to extract from 1 ID
        return int(np.ceil(len(self.indices) / self.batch_size))

In [9]:
train_dataset = TimeDataset(data_3d, CFG.train_sequence_length, CFG.predict_sequence_length)
valid_dataset = TimeDataset(data_3d, CFG.train_sequence_length, CFG.predict_sequence_length)

print(train_dataset[0][0].shape)
print(train_dataset[0][1].shape)

(64, 12, 3)
(64, 3)


In [10]:
def build_model():
    inputs = tf.keras.Input(shape=(CFG.train_sequence_length, 3))
    
    config = AutoConfig()("rnn")
    config.rnn_type = "lstm"
    backbone = AutoModel.from_config(config=config)
    
    outputs = backbone(inputs)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(), metrics = ['mae'])
    return model


model = build_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 12, 3)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ encoder (Encoder)                    │ [(None, 12, 64), (None,     │               0 │
│                                      │ 128)]                       │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             129 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 1, 1)                │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,153 (129.50 KB)

 Trainable params: 33,153 (129.50 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(train_dataset, validation_data=valid_dataset, epochs=10) 
model.save_weights('./sales_model.weights.h5')

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1747278246.661268      65 cuda_dnn.cc:529] Loaded cuDNN version 90300


282/282 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 350.8878 - mae: 350.8878 - val_loss: 182.2173 - val_mae: 182.2173
Epoch 2/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 170.9699 - mae: 170.9699 - val_loss: 173.9711 - val_mae: 173.9711
Epoch 3/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 177.6421 - mae: 177.6421 - val_loss: 151.1356 - val_mae: 151.1356
Epoch 4/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 167.8778 - mae: 167.8778 - val_loss: 181.6729 - val_mae: 181.6729
Epoch 5/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 200.4965 - mae: 200.4965 - val_loss: 161.8659 - val_mae: 161.8659
Epoch 6/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 159.5687 - mae: 159.5687 - val_loss: 203.8276 - val_mae: 203.8276
Epoch 7/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 191.8186 - mae: 191.8186 - val_loss: 168.8145 - val_mae: 168.8145
Epoch 8/10
282/282 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 154.9041 - mae: 154.9041 - val_loss: 148.9643 - val_mae: 148.